In [ ]:
!pip3 install pytelegrambotapi

In [ ]:
!pip3 install natasha

In [ ]:
import random
import pandas as pd
import datetime
import telebot
from telebot import types


bot = telebot.TeleBot('5050852838:AAGEeJwfdGCQbGx9jV5UvOydTaapqVkCcSY')

In [ ]:
import requests
import json
import time

In [ ]:
from natasha import (Segmenter,
                     NewsEmbedding,
                     NewsMorphTagger,
                     NewsSyntaxParser,
                     NewsNERTagger,
                     Doc)
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

from natasha import MorphVocab
from natasha import DatesExtractor
from natasha import NewsNERTagger

Определение города для показа погоды:

In [ ]:
def chosen_city(text):
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  doc.parse_syntax(syntax_parser)
  doc.tag_ner(ner_tagger)

  natasha_result = [str(doc.spans[i].text.lower()) for i in range(len(doc.spans))]

  moscow = ['москв', 'мск']
  spb = ['спб', 'ленинград', 'петербург', 'питер']
  text = text.lower()
  city = None
  count = 0
  for city1 in spb:
    for city2 in moscow:

      # check whether a user has typed two cities simultaneously
      if (city1 in text or city1 in natasha_result) and (city2 in text or city2 in natasha_result):
        count += 1
  
      if (city1 in text or city1 in natasha_result):
        city = 'Санкт-Петербурге'
      elif (city2 in text or city2 in natasha_result):
        city = 'Москве'


  if count > 0:
    return None
  else:
    return city



Определяем, на какой день показывать погоду:

In [ ]:
def chosen_date(text):
  text = text.lower()
  doc = Doc(text)

  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  doc.parse_syntax(syntax_parser)

  morph_vocab = MorphVocab()
  dates_extractor = DatesExtractor(morph_vocab)

  today = datetime.date.today()
  tomorrow = today + datetime.timedelta(days=1)

  today = "day={}, month={}".format(today.day, today.month)
  tomorrow = "day={}, month={}".format(tomorrow.day, tomorrow.month)
  res = list(dates_extractor(text))
  res_day_month = ['day={}, month={}'.format(res[i].fact.day, res[i].fact.month) for i in range(len(res))]

  if 'завтра' in text or tomorrow in res_day_month:
    return 'завтра'
  elif 'сейчас' in text or 'момент' in text:
    return 'сейчас'
  elif 'сегодня' in text or today in res_day_month:
    return 'сегодня'
  else:
    return None

In [ ]:
def greeting():
	greeting = ['Привет! Хочешь узнать погоду в Москве или Санкт-Петербурге? Тогда укажи город и время (сейчас, сегодня завтра).', 
             'Привет, давно не виделись! Хочешь узнать погоду в Москве или Санкт-Петербурге? Тогда укажи город и время (сейчас, сегодня завтра).',
             'Доброго времени суток! Хочешь узнать погоду в Москве или Санкт-Петербурге? Тогда укажи город и время (сейчас, сегодня завтра).',
             'Здравствуй! Хочешь узнать погоду в Москве или Санкт-Петербурге? Тогда укажи город и время (сейчас, сегодня завтра).']

	return random.choice(greeting)

In [ ]:
def bye():
  bye = ['Хорошего дня!', 'Увидимся!', 'До скорого!', 'Пока-пока :)']
  return random.choice(bye)

In [ ]:
def show_error():
	show_error =  ['Я не понимаю. Не мог бы ты немножко уточнить город и время?', 
                'Мне кажется, что я не совсем тебя понимаю :( Не мог бы ты немного уточнить запрос, а именно: город и время?',
                'Я не могу уловить твои намерения. Не мог бы ты дать мне несколько пояснений о городе и времени, пожалуйста?']

	return random.choice(show_error) + " Ты хочешь узнать прогноз погоды? Я рад рассказать о погоде в Москве или Санкт-Петербурге! Выбери город и время (сейчас, сегодня, завтра)."

Функция работы с API погоды:

In [ ]:
WEATHER_TOKEN = 'a3caef32ad37d5e88e6327d45dba6467'

WEATHER_URL = 'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=hourly&appid={token}&units={units}'

def get_weather(location, time):
  if location == 'Санкт-Петербурге':
    url = WEATHER_URL.format(lat=59.93, lon=30.36,
                                   token=WEATHER_TOKEN, units='metric')
  elif location == 'Москве':
    url = WEATHER_URL.format(lat=55.75, lon=37.61,
                                   token=WEATHER_TOKEN, units='metric')
  else:
    return None
  response = requests.get(url)
  data = json.loads(response.content)
  return data

Главная функция работы с тектом:

In [ ]:
def main(text):
  city = chosen_city(text)
  text = text.lower()

  for word in ['прив', 'утро', 'день', 'вечер', 'здравствуй']:
    if word in text:
      return greeting()
      
  for word in ['пока', 'встречи', 'свидани', 'спокойной']:
    if word in text:
      return bye()
    
  data = get_weather(city, chosen_date(text))
  if city is not None and data is not None:
    if chosen_date(text) == 'сейчас':
      temp = data['current']['temp']
      msg = f'Температура в {city} сейчас {temp}°С.'
      return msg
    if chosen_date(text) == 'сегодня':
      temp_day = data['daily'][0]['temp']['day']
      temp_night = data['daily'][0]['temp']['night']
      msg = f'Сегодня температура в {city} в течение дня {temp_day}°С, в течение ночи {temp_night}°С.'
      return msg
    elif chosen_date(text) == 'завтра':
      temp_day = data['daily'][1]['temp']['day']
      temp_night = data['daily'][1]['temp']['night']
      msg = f'Завтра температура в {city} в течение дня будет {temp_day}°С, в течение ночи - {temp_night}°С.'
      return msg
    else:
      return 'Не мог бы ты уточнить день, для которого нужно определить погоду? А также, укажи, пожалуйста, еще раз город.'
  elif (chosen_date(text) == 'сегодня' or chosen_date(text) == 'завтра' or chosen_date(text) == 'сейчас'):
    return 'Не мог бы ты уточнить город, для которого нужно определить погоду? А также, укажи, пожалуйста, еще раз день.'
  elif city is None and chosen_date(text) != 'сегодня' and chosen_date(text) != 'завтра' and chosen_date(text) != 'сейчас':
    return 'Не мог бы ты уточнить город и день, чтобы я смог точнее определить погоду?'

      
  return show_error()

In [ ]:
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, 'Привет! Я могу подсказать тебе погоду в Москве и Санкт-Петербурге на данный момент, сегодня и завтра.')

In [ ]:
@bot.message_handler(commands=['help'])
def welcome_help(message):
    bot.send_message(message.chat.id, 'Выбери город (Санкт-Петербург, Москва) и время (сейчас, сегодня, завтра), чтобы узнать погоду. ')

In [ ]:
@bot.message_handler(content_types=['text'])
def get_messages(message):
	bot.send_message(message.from_user.id, main(message.text))

In [ ]:
bot.polling(none_stop=True, interval=0)